## Sumy Algorithms

* Using Latent Semantic Analysis
* Using LexRank approach
* Using TextRank approach
* Using SumBasic approach
* Using KL-sum

### Requirements

* fuzzy wuzzy

In [1]:
#!pip install fuzzywuzzy[speedup]
import pandas as pd
import en_core_web_sm
import numpy as np
from utils import load_espn, save_espn
path = r'C:/Users/atenk/Documents/ISM/HeadlineGeneration/ESPN_football_2.csv'
df = load_espn(path)
from fuzzywuzzy import process, fuzz

In [2]:
df.head()

,author,class,data-id,sport,teamname,timestamp,url,summary,headline,rawText,...,LSASummarizer,LexRankSummarizer,TextRankSummarizer,SumBasicSummarizer,KLSumSummarizer,base_url,LSAFuzzy,LexRankFuzzy,TextRankFuzzy,SumBasicFuzzy
0,Mike Rodak,story-link,buffalo-bills-33083,nfl,buffalo-bills,3d,http://espn.com/blog/buffalo-bills/post/_/id/3...,"Andre Reed, who ranks seventh in NFL history w...",Andre Reed has acting bug after cameos in 'Mac...,"Andre Reed, who ranks seventh in NFL history w...",...,0,After a successful spot on the show in January...,11,"""Everything is timed,"" Reed said.",9,http://espn.com/blog,17,5,11,14
1,Todd McShay,story-link,26489910,nfl,buffalo-bills,4d,http://insider.espn.com/nfl/draft2019/insider/...,Todd McShay hears that the Giants might not be...,Todd McShay's top five 2019 NFL draft needs fo...,Todd McShay hears that the Giants might not be...,...,6,The Bills have their quarterback in Josh Allen...,"Top draft needs: TE, WR, DT, EDGE, QB","Top draft needs: QB, G, OT, C, EDGE","Top draft needs: TE, WR, DT, EDGE, QB",http://insider.espn.com/nfl,6,5,14,0
2,ESPN.com,story-link,26473482,nfl,buffalo-bills,6d,http://www.espn.com/nfl/draft2019/story/_/id/2...,"The NFL draft annually brings back memories, a...",NFL draft do-overs: Let's re-pick ... and fix ...,"The NFL draft annually brings back memories, a...",...,"Sure, just about every team in the NFL would l...",10 pick),53,10 pick),First round of the 2016 draft (21st overall)Ac...,http://www.espn.com/nfl,145,65,53,65
3,T.J. Berka,story-link,24367000,nfl,buffalo-bills,7d,http://www.espn.com/nhl/story/_/id/24367000/th...,Since we updated the sports misery index in De...,Sports misery index: Most miserable fan bases ...,Since we updated the sports misery index in De...,...,The Brewers were the focus of this article whe...,189,130,14,96,http://www.espn.com/nhl,150,189,130,14
4,Mike Rodak,story-link,26426098,nfl,buffalo-bills,13d,http://www.espn.com/nfl/story/_/id/26426098/in...,Former NFL offensive lineman Richie Incognito ...,Incognito pleads guilty to disorderly conduct,Former NFL offensive lineman Richie Incognito ...,...,Appearing on a podcast hosted by former Bills ...,12,12,12,Appearing on a podcast hosted by former Bills ...,http://www.espn.com/nfl,6,12,12,12


In [3]:
def lsa_summarize (url, text):
    from sumy.parsers.html import HtmlParser
    from sumy.nlp.tokenizers import Tokenizer
    from sumy.summarizers.lsa import LsaSummarizer as Summarizer
    from sumy.nlp.stemmers import Stemmer
    from sumy.utils import get_stop_words
    
    LANGUAGE = "english"
    SENTENCES_COUNT = 1

    parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)
    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    sent = "";
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        sent += str(sentence)
    try:
        return text.index(sent)
    except:
        return sent

In [17]:
def lsa(df):
    df['LSASummarizer'] = lsa_summarize(df['url'], df['text'])
    return df
df = df.apply(lsa, axis=1)

In [20]:
def lsa_fuzzy (url, text):
    from sumy.parsers.html import HtmlParser
    from sumy.nlp.tokenizers import Tokenizer
    from sumy.summarizers.lsa import LsaSummarizer as Summarizer
    from sumy.nlp.stemmers import Stemmer
    from sumy.utils import get_stop_words
    
    LANGUAGE = "english"
    SENTENCES_COUNT = 1

    parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)
    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    sent = "";
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        sent += str(sentence)
    try:
        return text.index(sent)
    except:
        return text.index(process.extract(sent, text)[0][0])

In [21]:
def lsaf(df):
    df['LSAFuzzy'] = lsa_fuzzy(df['url'], df['text'])
    return df
df = df.apply(lsaf, axis=1)

In [30]:
def lexrank_summarize (url, text):
    from sumy.parsers.html import HtmlParser
    from sumy.nlp.tokenizers import Tokenizer
    from sumy.summarizers.lex_rank import LexRankSummarizer as Summarizer
    from sumy.nlp.stemmers import Stemmer
    from sumy.utils import get_stop_words

    LANGUAGE = "english"
    SENTENCES_COUNT = 1

    parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)
    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    sent = "";
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        sent += str(sentence)
    try:
        return text.index(sent)
    except:
        return sent

In [31]:
def lrs(df):
    df['LexRankSummarizer'] = lexrank_summarize(df['url'], df['text'])
    return df
df = df.apply(lrs, axis=1)

In [23]:
def lexrank_fuzzy (url, text):
    from sumy.parsers.html import HtmlParser
    from sumy.nlp.tokenizers import Tokenizer
    from sumy.summarizers.lex_rank import LexRankSummarizer as Summarizer
    from sumy.nlp.stemmers import Stemmer
    from sumy.utils import get_stop_words

    LANGUAGE = "english"
    SENTENCES_COUNT = 1

    parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)
    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    sent = "";
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        sent += str(sentence)
    try:
        return text.index(sent)
    except:
        return text.index(process.extract(sent, text)[0][0])

In [24]:
def lrsf(df):
    df['LexRankFuzzy'] = lexrank_fuzzy(df['url'], df['text'])
    return df
df = df.apply(lrsf, axis=1)

In [35]:
def textrank_summarize (url, text):
    from sumy.parsers.html import HtmlParser
    from sumy.nlp.tokenizers import Tokenizer
    from sumy.summarizers.text_rank import TextRankSummarizer as Summarizer
    from sumy.nlp.stemmers import Stemmer
    from sumy.utils import get_stop_words

    LANGUAGE = "english"
    SENTENCES_COUNT = 1

    parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)
    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    sent = "";
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        sent += str(sentence)
    try:
        return text.index(sent)
    except:
        return sent

In [40]:
def trs(df):
    df['TextRankSummarizer'] = textrank_summarize(df['url'], df['text'])
    return df
df = df.apply(trs, axis=1)

In [25]:
def textrank_fuzzy (url, text):
    from sumy.parsers.html import HtmlParser
    from sumy.nlp.tokenizers import Tokenizer
    from sumy.summarizers.text_rank import TextRankSummarizer as Summarizer
    from sumy.nlp.stemmers import Stemmer
    from sumy.utils import get_stop_words

    LANGUAGE = "english"
    SENTENCES_COUNT = 1

    parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)
    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    sent = "";
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        sent += str(sentence)
    try:
        return text.index(sent)
    except:
        return text.index(process.extract(sent, text)[0][0])

In [26]:
def trsf(df):
    df['TextRankFuzzy'] = textrank_fuzzy(df['url'], df['text'])
    return df
df = df.apply(trsf, axis=1)

In [3]:
def sumbasic_summarize (url, text):
    from sumy.parsers.html import HtmlParser
    from sumy.nlp.tokenizers import Tokenizer
    from sumy.summarizers.sum_basic import SumBasicSummarizer as Summarizer
    from sumy.nlp.stemmers import Stemmer
    from sumy.utils import get_stop_words

    LANGUAGE = "english"
    SENTENCES_COUNT = 1

    parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)
    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    sent = "";
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        sent += str(sentence)
    try:
        return text.index(sent)
    except:
        return sent

In [7]:
def sbs(df):
    df['SumBasicSummarizer'] = sumbasic_summarize(df['url'], df['text'])
    return df
df = df.apply(sbs, axis=1)

In [27]:
def sumbasic_fuzzy (url, text):
    from sumy.parsers.html import HtmlParser
    from sumy.nlp.tokenizers import Tokenizer
    from sumy.summarizers.sum_basic import SumBasicSummarizer as Summarizer
    from sumy.nlp.stemmers import Stemmer
    from sumy.utils import get_stop_words

    LANGUAGE = "english"
    SENTENCES_COUNT = 1

    parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)
    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    sent = "";
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        sent += str(sentence)
    try:
        return text.index(sent)
    except:
        return text.index(process.extract(sent, text)[0][0])

In [28]:
def sbsf(df):
    df['SumBasicFuzzy'] = sumbasic_fuzzy(df['url'], df['text'])
    return df
df = df.apply(sbsf, axis=1)

In [15]:
def klsum_summarize (url, text):
    from sumy.parsers.html import HtmlParser
    from sumy.nlp.tokenizers import Tokenizer
    from sumy.summarizers.kl import KLSummarizer as Summarizer
    from sumy.nlp.stemmers import Stemmer
    from sumy.utils import get_stop_words

    LANGUAGE = "english"
    SENTENCES_COUNT = 1

    parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)
    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    sent = "";
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        sent += str(sentence)
    try:
        return text.index(sent)
    except:
        return sent

In [ ]:
def kls(df):
    df['KLSumSummarizer'] = klsum_summarize(df['url'], df['text'])
    return df
df = df.apply(kls, axis=1)

In [26]:
def klsum_fuzzy (text, KLSummary):
    if KLSummary.isdigit():
        return KLSummary
    else:
        return text.index(process.extract(KLSummary, text)[0][0])

In [27]:
def klsf(df):
    df['KLSumFuzzy'] = klsum_fuzzy(df['text'], df['KLSumSummarizer'])
    return df
df = df.apply(klsf, axis=1)

In [30]:
df.head()

,author,class,data-id,sport,teamname,timestamp,url,summary,headline,rawText,...,LexRankSummarizer,TextRankSummarizer,SumBasicSummarizer,KLSumSummarizer,base_url,LSAFuzzy,LexRankFuzzy,TextRankFuzzy,SumBasicFuzzy,KLSumFuzzy
0,Mike Rodak,story-link,buffalo-bills-33083,nfl,buffalo-bills,3d,http://espn.com/blog/buffalo-bills/post/_/id/3...,"Andre Reed, who ranks seventh in NFL history w...",Andre Reed has acting bug after cameos in 'Mac...,"Andre Reed, who ranks seventh in NFL history w...",...,After a successful spot on the show in January...,11,"""Everything is timed,"" Reed said.",9,http://espn.com/blog,17,5,11,14,9
1,Todd McShay,story-link,26489910,nfl,buffalo-bills,4d,http://insider.espn.com/nfl/draft2019/insider/...,Todd McShay hears that the Giants might not be...,Todd McShay's top five 2019 NFL draft needs fo...,Todd McShay hears that the Giants might not be...,...,The Bills have their quarterback in Josh Allen...,"Top draft needs: TE, WR, DT, EDGE, QB","Top draft needs: QB, G, OT, C, EDGE","Top draft needs: TE, WR, DT, EDGE, QB",http://insider.espn.com/nfl,6,5,14,0,14
2,ESPN.com,story-link,26473482,nfl,buffalo-bills,6d,http://www.espn.com/nfl/draft2019/story/_/id/2...,"The NFL draft annually brings back memories, a...",NFL draft do-overs: Let's re-pick ... and fix ...,"The NFL draft annually brings back memories, a...",...,10 pick),53,10 pick),First round of the 2016 draft (21st overall)Ac...,http://www.espn.com/nfl,145,65,53,65,5
3,T.J. Berka,story-link,24367000,nfl,buffalo-bills,7d,http://www.espn.com/nhl/story/_/id/24367000/th...,Since we updated the sports misery index in De...,Sports misery index: Most miserable fan bases ...,Since we updated the sports misery index in De...,...,189,130,14,96,http://www.espn.com/nhl,150,189,130,14,96
4,Mike Rodak,story-link,26426098,nfl,buffalo-bills,13d,http://www.espn.com/nfl/story/_/id/26426098/in...,Former NFL offensive lineman Richie Incognito ...,Incognito pleads guilty to disorderly conduct,Former NFL offensive lineman Richie Incognito ...,...,12,12,12,Appearing on a podcast hosted by former Bills ...,http://www.espn.com/nfl,6,12,12,12,6


In [31]:
save_espn(df, path)